In [106]:
import csv
import math

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

RANDOM_SEED = 42

# Specify paths

In [107]:
dataset = 'model/dynamic_classifier/dynamic_data.csv'
model_save_path = 'model/dynamic_classifier/dynamic_classifier.hdf5'
tflite_save_path = 'model/static_classifier/static_classifier.tflite'

# Set number of classes

In [108]:
NUM_CLASSES = 2

# Input Length

In [109]:
TIME_STEPS = 57
DIMENSION = 2

# Dataset reading

In [110]:
dynamic_gestures = [
    'none',
    'click'
]

data = np.concatenate([
    np.load('dataset/seq_none_1636955394.npy'),
    np.load('dataset/seq_click_1636955394.npy')
], axis=0)

In [111]:
X_dataset = data[:, :, :-1]
X_dataset.shape

(200, 57, 99)

In [112]:
labels = data[:, 0, -1]
y_dataset = to_categorical(labels, num_classes=len(dynamic_gestures))
y_dataset.shape

(200, 2)

In [119]:
X_dataset = X_dataset.astype(np.float32)
y_dataset = y_dataset.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, test_size=0.2, random_state=RANDOM_SEED)
print(X_train.shape)
print(y_train.shape)

(160, 57, 99)
(160, 2)


# Model building

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=X_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(dynamic_gestures), activation='softmax')
])

In [121]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 64)                41984     
_________________________________________________________________
dense_22 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 66        
Total params: 44,130
Trainable params: 44,130
Non-trainable params: 0
_________________________________________________________________


In [122]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
# Callback for early stopping
es_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')

In [123]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['acc']
)

# Model training

In [124]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
2/2 [==============================] - 1s 187ms/step - loss: 93.6549 - acc: 0.5427 - val_loss: 79.6970 - val_acc: 0.4500

Epoch 00001: val_acc improved from -inf to 0.45000, saving model to model/dynamic_classifier\dynamic_classifier.hdf5
Epoch 2/1000
2/2 [==============================] - 0s 43ms/step - loss: 107.7367 - acc: 0.5458 - val_loss: 41.7732 - val_acc: 0.6250

Epoch 00002: val_acc improved from 0.45000 to 0.62500, saving model to model/dynamic_classifier\dynamic_classifier.hdf5
Epoch 3/1000
2/2 [==============================] - 0s 38ms/step - loss: 96.9327 - acc: 0.5438 - val_loss: 63.6900 - val_acc: 0.5750

Epoch 00003: val_acc did not improve from 0.62500
Epoch 4/1000
2/2 [==============================] - 0s 39ms/step - loss: 101.0697 - acc: 0.5286 - val_loss: 53.9813 - val_acc: 0.5750

Epoch 00004: val_acc did not improve from 0.62500
Epoch 5/1000
2/2 [==============================] - 0s 38ms/step - loss: 92.2610 - acc: 0.5380 - val_loss: 54.7790 - val_acc

2/2 [==============================] - 0s 39ms/step - loss: 18.2007 - acc: 0.5812 - val_loss: 28.8409 - val_acc: 0.4250

Epoch 00043: val_acc did not improve from 0.70000
Epoch 44/1000
2/2 [==============================] - 0s 44ms/step - loss: 20.5164 - acc: 0.5276 - val_loss: 48.8755 - val_acc: 0.3500

Epoch 00044: val_acc did not improve from 0.70000
Epoch 45/1000
2/2 [==============================] - 0s 38ms/step - loss: 23.4120 - acc: 0.5844 - val_loss: 26.0179 - val_acc: 0.5000

Epoch 00045: val_acc did not improve from 0.70000
Epoch 46/1000
2/2 [==============================] - 0s 38ms/step - loss: 18.7768 - acc: 0.5380 - val_loss: 16.6576 - val_acc: 0.4750

Epoch 00046: val_acc did not improve from 0.70000
Epoch 47/1000
2/2 [==============================] - 0s 38ms/step - loss: 20.6317 - acc: 0.4943 - val_loss: 54.5802 - val_acc: 0.3250

Epoch 00047: val_acc did not improve from 0.70000
Epoch 48/1000
2/2 [==============================] - 0s 38ms/step - loss: 21.0086 - acc: 

2/2 [==============================] - 0s 38ms/step - loss: 11.0699 - acc: 0.5974 - val_loss: 20.7363 - val_acc: 0.4250

Epoch 00087: val_acc did not improve from 0.70000
Epoch 88/1000
2/2 [==============================] - 0s 37ms/step - loss: 12.4682 - acc: 0.5766 - val_loss: 16.2342 - val_acc: 0.5500

Epoch 00088: val_acc did not improve from 0.70000
Epoch 89/1000
2/2 [==============================] - 0s 38ms/step - loss: 9.7459 - acc: 0.5812 - val_loss: 17.0687 - val_acc: 0.5750

Epoch 00089: val_acc did not improve from 0.70000

Epoch 00089: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 90/1000
2/2 [==============================] - 0s 39ms/step - loss: 11.9439 - acc: 0.6005 - val_loss: 19.2965 - val_acc: 0.4750

Epoch 00090: val_acc did not improve from 0.70000
Epoch 91/1000
2/2 [==============================] - 0s 39ms/step - loss: 10.4291 - acc: 0.6099 - val_loss: 20.8037 - val_acc: 0.4500

Epoch 00091: val_acc did not improve from 0.70000
Epoch 92/

2/2 [==============================] - 0s 111ms/step - loss: 2.6718 - acc: 0.7146 - val_loss: 6.2778 - val_acc: 0.4750

Epoch 00131: val_acc did not improve from 0.70000
Epoch 132/1000
2/2 [==============================] - 0s 37ms/step - loss: 3.3018 - acc: 0.7135 - val_loss: 5.5243 - val_acc: 0.5000

Epoch 00132: val_acc did not improve from 0.70000
Epoch 133/1000
2/2 [==============================] - 0s 38ms/step - loss: 2.8742 - acc: 0.6807 - val_loss: 5.2826 - val_acc: 0.6000

Epoch 00133: val_acc did not improve from 0.70000
Epoch 134/1000
2/2 [==============================] - 0s 41ms/step - loss: 3.4153 - acc: 0.6646 - val_loss: 5.9683 - val_acc: 0.4750

Epoch 00134: val_acc did not improve from 0.70000
Epoch 135/1000
2/2 [==============================] - 0s 42ms/step - loss: 2.4694 - acc: 0.7484 - val_loss: 6.6224 - val_acc: 0.4750

Epoch 00135: val_acc did not improve from 0.70000
Epoch 136/1000
2/2 [==============================] - 0s 42ms/step - loss: 3.0035 - acc: 0.703

2/2 [==============================] - 0s 40ms/step - loss: 2.0064 - acc: 0.7542 - val_loss: 4.9267 - val_acc: 0.5250

Epoch 00175: val_acc did not improve from 0.70000
Epoch 176/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.8767 - acc: 0.8083 - val_loss: 4.7782 - val_acc: 0.5250

Epoch 00176: val_acc did not improve from 0.70000
Epoch 177/1000
2/2 [==============================] - 0s 40ms/step - loss: 1.8482 - acc: 0.7349 - val_loss: 4.7945 - val_acc: 0.5250

Epoch 00177: val_acc did not improve from 0.70000
Epoch 178/1000
2/2 [==============================] - 0s 40ms/step - loss: 1.8016 - acc: 0.7729 - val_loss: 4.8223 - val_acc: 0.5250

Epoch 00178: val_acc did not improve from 0.70000
Epoch 179/1000
2/2 [==============================] - 0s 40ms/step - loss: 1.9119 - acc: 0.7844 - val_loss: 4.8099 - val_acc: 0.5250

Epoch 00179: val_acc did not improve from 0.70000
Epoch 180/1000
2/2 [==============================] - 0s 66ms/step - loss: 1.8653 - acc: 0.7786

...

2/2 [==============================] - 0s 38ms/step - loss: 1.4123 - acc: 0.7974 - val_loss: 3.9477 - val_acc: 0.5500

Epoch 00792: val_acc did not improve from 0.70000
Epoch 793/1000
2/2 [==============================] - 0s 40ms/step - loss: 1.3681 - acc: 0.8000 - val_loss: 3.9477 - val_acc: 0.5500

Epoch 00793: val_acc did not improve from 0.70000
Epoch 794/1000
2/2 [==============================] - 0s 41ms/step - loss: 1.3695 - acc: 0.7948 - val_loss: 3.9477 - val_acc: 0.5500

Epoch 00794: val_acc did not improve from 0.70000
Epoch 795/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.3742 - acc: 0.7974 - val_loss: 3.9477 - val_acc: 0.5500

Epoch 00795: val_acc did not improve from 0.70000
Epoch 796/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4274 - acc: 0.7974 - val_loss: 3.9477 - val_acc: 0.5500

Epoch 00796: val_acc did not improve from 0.70000
Epoch 797/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.3156 - acc: 0.7948


Epoch 00836: val_acc did not improve from 0.70000
Epoch 837/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4144 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00837: val_acc did not improve from 0.70000
Epoch 838/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4079 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00838: val_acc did not improve from 0.70000
Epoch 839/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.3946 - acc: 0.7922 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00839: val_acc did not improve from 0.70000

Epoch 00839: ReduceLROnPlateau reducing learning rate to 1.525878978725359e-08.
Epoch 840/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.3241 - acc: 0.8026 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00840: val_acc did not improve from 0.70000
Epoch 841/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.3752 - acc: 0.8026 - val_loss: 3.9475 - val_acc: 0.5500

2/2 [==============================] - 0s 38ms/step - loss: 1.4648 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00881: val_acc did not improve from 0.70000
Epoch 882/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4308 - acc: 0.7870 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00882: val_acc did not improve from 0.70000
Epoch 883/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.4050 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00883: val_acc did not improve from 0.70000
Epoch 884/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.4250 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00884: val_acc did not improve from 0.70000
Epoch 885/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.4575 - acc: 0.7870 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00885: val_acc did not improve from 0.70000
Epoch 886/1000
2/2 [==============================] - 0s 107ms/step - loss: 1.4428 - acc: 0.789

2/2 [==============================] - 0s 39ms/step - loss: 1.4640 - acc: 0.7844 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00925: val_acc did not improve from 0.70000
Epoch 926/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.3557 - acc: 0.7974 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00926: val_acc did not improve from 0.70000
Epoch 927/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.3992 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00927: val_acc did not improve from 0.70000
Epoch 928/1000
2/2 [==============================] - 0s 40ms/step - loss: 1.3410 - acc: 0.7974 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00928: val_acc did not improve from 0.70000
Epoch 929/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4215 - acc: 0.7896 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00929: val_acc did not improve from 0.70000
Epoch 930/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4526 - acc: 0.7870

2/2 [==============================] - 0s 38ms/step - loss: 1.3305 - acc: 0.7922 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00969: val_acc did not improve from 0.70000
Epoch 970/1000
2/2 [==============================] - 0s 37ms/step - loss: 1.3806 - acc: 0.7974 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00970: val_acc did not improve from 0.70000
Epoch 971/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4353 - acc: 0.7844 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00971: val_acc did not improve from 0.70000
Epoch 972/1000
2/2 [==============================] - 0s 39ms/step - loss: 1.4144 - acc: 0.7922 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00972: val_acc did not improve from 0.70000
Epoch 973/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.3703 - acc: 0.7948 - val_loss: 3.9475 - val_acc: 0.5500

Epoch 00973: val_acc did not improve from 0.70000
Epoch 974/1000
2/2 [==============================] - 0s 38ms/step - loss: 1.4101 - acc: 0.7948

In [125]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [126]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[1.2474088e-13 1.0000000e+00]
1


# Confusion matrix

In [127]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

TypeError: unhashable type: 'numpy.ndarray'

# Convert to model for Tensorflow-Lite

In [128]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)
model = tf.keras.models.load_model(model_save_path)

In [129]:
tflite_save_path = 'model/dynamic_classifier/dynamic_classifier.tflite'

In [130]:
# Transform (quantize) model
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\jerem\AppData\Local\Temp\tmpuc6we384\assets


58288

# Inference test

In [131]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [132]:
# Get I/O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

[{'name': 'lstm_8_input', 'index': 0, 'shape': array([ 1, 57, 99]), 'shape_signature': array([-1, 57, 99]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [133]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [134]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

Wall time: 2.99 ms


In [135]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[1.1955772e-26 1.0000000e+00]
1
